In [ ]:
import os

import cartopy.crs as ccrs
import gpxpy
import gpxpy.gpx

from bs4 import BeautifulSoup as bs

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
data_dir = './data/'
save_figs = False
output_dir = './outputs/'

In [ ]:
def get_files_in_dir(directory, return_dirs=False, verbose=False):
    if not os.path.exists(directory):
        raise Exception(f"{directory} does not exist!")
    files, dirs = None, None
    for (path, dirs, files) in os.walk(directory):
        if verbose:
            print("path: ", path)
            print("dirs", dirs)
            print("files")
            for i, file in enumerate(files):
                print("\t", i, file)
        break
    return files if not return_dirs else (files, dirs)

In [ ]:
files = get_files_in_dir(data_dir)
files = [f for f in files if '.gpx' in f[-4:]]
files

In [ ]:
stepkeys = [f[f.find('.')-2:-4].strip()
            .split(' ')[0]
            for f in files]
step2file_long = {
    k: f for k, f in zip(stepkeys, files)
}

step2file_long


In [ ]:
sk = '0.1'
step2file_long[sk]

In [ ]:
step2file_long.keys()

In [ ]:
# Drop circle tours in cities, aka not steps
step2file = {
    k: step2file_long[k] for k in step2file_long if '.' in k
}
step2file.keys()

In [ ]:
sk = '0.1'
step2file[sk]

In [ ]:
len(step2file)

In [ ]:
def load_gpx_file(file_path:str):
    if not os.path.exists(file_path):
        raise Exception(f"{file_path} does not exist!")
    with open(file_path, 'r') as fo:
        gpx_file = gpxpy.parse(fo)
    return gpx_file

In [ ]:
sk = '0.1'
gpx_f = load_gpx_file(data_dir + step2file[sk])
gpx_f

In [ ]:
# if you wanted to use BeautifulSoup for parsing XML...
gpx_content = bs(gpx_f.to_xml(), features='xml')
gpx_content.metadata

In [ ]:
step_max = max([
    int(k.split('.')[0]) for k in step2file.keys()
])
step_max

In [ ]:
to_skip = ['16.1'] # 16.1 was within Malaga => no recording

In [ ]:
step = 0
step2gpx = {}
while step < step_max + 1:
    substep = 1
    while True:
        sk = f"{step}.{substep}"

        if sk in to_skip:
            substep += 1
            continue
        if sk not in step2file:
            break

        gpx = load_gpx_file(data_dir + step2file[sk])
        step2gpx[sk] = gpx

        print(
            sk, len(gpx.tracks[0].segments[0].points)
        )  # there is one track, inside of it is one segment, inside of which are the points

        substep += 1

    print(sk)

    step += 1


In [ ]:
gpx_all = gpxpy.gpx.GPX()
gpx_all.name = "Hateless Bikemore Finland Morocco 2022"

# Create first track in our GPX:
gpx_track = gpxpy.gpx.GPXTrack()
gpx_all.tracks.append(gpx_track)

# add all the segments to our GPX track:
step = 0
while step < step_max + 1:
    substep = 1
    while True:
        sk = f"{step}.{substep}"
        if sk in to_skip:
            substep += 1
            continue
        if sk not in step2file:
            break

        gpx = step2gpx[sk]
        gpx_track.segments.append(gpx.tracks[0].segments[0])

        substep += 1

    print(sk)

    step += 1


In [ ]:
gpx_all.get_moving_data(raw=False)

In [ ]:
gpx_all.get_uphill_downhill()

In [ ]:
gpx_all.get_points_no()

In [ ]:
gpx_all.get_bounds()

In [ ]:
fname = data_dir + 'processed_gpx/finland_to_morocco_5mon.gpx'
with open(fname, 'w') as fo:
    fo.write(gpx_all.to_xml())
    print('saved concatenated XML file to ', fname)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
points = [
    [[p.latitude, p.longitude, p.elevation, p.time] for p in segment.points]
    for segment in gpx_all.tracks[0].segments
]
points = np.concatenate(points)
len(points)

In [ ]:
df_all = pd.DataFrame(points, columns=['latitude', 'longitude', 'elevation', 'time'])
df_all.head()

In [ ]:
fname = data_dir + 'processed_gpx/finland_to_morocco_5mon.csv'
df_all.to_csv(fname)

In [ ]:

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

ax = plt.axes(projection=ccrs.EuroPP())
ax.stock_img()
plt.show()


In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

plt.show()